In [1]:
import tensorflow as tf
import cv2
import PIL
import os 
from PIL import Image
import glob
import numpy as np

Init Plugin
Init Graph Optimizer
Init Kernel


In [2]:
cat = glob.glob('train/*cat*')

In [3]:
dog = glob.glob('train/*dog*')

In [4]:
print(len(dog),len(cat))

12500 12500


In [5]:
a=cv2.imread(cat[0])

In [6]:
a = cv2.resize(a,(255,255))

In [7]:
animal_images_dict = {
    'cat':cat,
    'dog':dog
}

animal_labels_dict = {
    'cat':0,
    'dog':1
}

In [8]:
X=[]
y=[]
for animal,images in animal_images_dict.items():
    for image in images:
        image = cv2.imread(image)
        image = cv2.resize(image,(224,224))
        X.append(image)
        y.append(animal_labels_dict[animal])

In [9]:
X = np.array(X)
y = np.array(y)

In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,shuffle=True)

In [11]:
X_train = X_train[0:3200]
X_test = X_test[0:800]
y_train = y_train[0:3200]
y_test = y_test[0:800]

In [12]:
# X_train = tf.convert_to_tensor(X_train)
# X_test = tf.convert_to_tensor(X_test)
# y_train = tf.convert_to_tensor(y_train)
# y_test = tf.convert_to_tensor(y_test)


In [13]:
X_train = tf.keras.applications.resnet50.preprocess_input(
    X_train, data_format=None
)
X_test = tf.keras.applications.resnet50.preprocess_input(
    X_test, data_format=None
)

In [14]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import resnet50

In [15]:
base_model = resnet50.ResNet50(include_top=False,weights='imagenet',classes=2)
base_model.trainable = False

Metal device set to: Apple M1


2022-07-27 17:49:16.869925: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-27 17:49:16.870286: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [18]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,TensorBoard
import os
file_path = 'model'
check_points_path = 'checkpoints/'
log_path = 'logs'
callbacks = ModelCheckpoint(filepath=check_points_path,save_weights_only=True,verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=3)
tensorboard = TensorBoard(log_dir=log_path)

2022-07-27 18:02:32.464542: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2022-07-27 18:02:32.464583: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2022-07-27 18:02:32.465015: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.


In [19]:

base_model = resnet50.ResNet50(include_top=False,weights='imagenet',classes=1000)
base_model.trainable = False
# with tf.device("/cpu:0"):
   

#         data_augmentation = keras.Sequential([
# #             layers.experimental.preprocessing.RandomZoom(0.3),
# #         layers.experimental.preprocessing.RandomRotation(0.1),
#             layers.experimental.preprocessing.RandomFlip(),
#         layers.experimental.preprocessing.RandomCrop(224,224),
#         ])


        
model = Sequential([
#     data_augmentation,
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Flatten(),
    layers.Dense(1024,activation='relu'),
    layers.Dense(1)
])
#         with tf.device("/gpu:0"):
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),metrics=['accuracy'])
model_history = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=50,callbacks=[callbacks,early_stop,tensorboard])

            

          

/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 1/50


2022-07-27 18:02:37.778786: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


  1/100 [..............................] - ETA: 2:56 - loss: 0.7977 - accuracy: 0.5312

2022-07-27 18:02:39.155119: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2022-07-27 18:02:39.155135: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.


  2/100 [..............................] - ETA: 1:30 - loss: 2.2784 - accuracy: 0.5156

2022-07-27 18:02:41.346358: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-07-27 18:02:41.346798: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2022-07-27 18:02:41.347837: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/train/plugins/profile/2022_07_27_18_02_41
2022-07-27 18:02:41.348444: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to logs/train/plugins/profile/2022_07_27_18_02_41/wangyuhuideMacBook-Pro.local.trace.json.gz
2022-07-27 18:02:41.349064: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/train/plugins/profile/2022_07_27_18_02_41
2022-07-27 18:02:41.349204: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for memory_profile.json.gz to logs/train/plugins/profile/2022_07_27_18_02_41/wangyuhuideMacBook-Pro.local.memory_profile.json.gz

100/100 [==============================] - ETA: 0s - loss: 0.1863 - accuracy: 0.9491

2022-07-27 18:03:15.841177: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


100/100 [==============================] - 48s 462ms/step - loss: 0.1863 - accuracy: 0.9491 - val_loss: 0.0367 - val_accuracy: 0.9837

Epoch 00001: saving model to checkpoints/
Epoch 2/50
100/100 [==============================] - 42s 420ms/step - loss: 0.0370 - accuracy: 0.9872 - val_loss: 0.0445 - val_accuracy: 0.9787

Epoch 00002: saving model to checkpoints/
Epoch 3/50
100/100 [==============================] - 42s 417ms/step - loss: 0.0176 - accuracy: 0.9934 - val_loss: 0.0384 - val_accuracy: 0.9850

Epoch 00003: saving model to checkpoints/
Epoch 4/50
100/100 [==============================] - 42s 422ms/step - loss: 0.0118 - accuracy: 0.9966 - val_loss: 0.0296 - val_accuracy: 0.9875

Epoch 00004: saving model to checkpoints/
Epoch 5/50
100/100 [==============================] - 42s 417ms/step - loss: 0.0053 - accuracy: 0.9991 - val_loss: 0.0300 - val_accuracy: 0.9887

Epoch 00005: saving model to checkpoints/
Epoch 6/50
100/100 [==============================] - 42s 416ms/step - 